In [1]:
import numpy as np
import math
import random
import time

In [2]:
room_w = 2000 #Width of the room in cm x
room_l = 2000 #Length of the room in cm y
room_h = 300 #Height of the room in cm z

detector_distance = 40 #How far the detector can 'see' in cm

chunk_size = 5 #The side length of each cube in cm

#Get Angle from Location

In [3]:
def get_robot_position_error(true_robot_position):
  x_error = (random.random() % 1000) - 500
  y_error = (random.random() % 1000) - 500
  z_error = (random.random() % 1000) - 500

  distance = np.sqrt((x_error ** 2) + (y_error ** 2) + (z_error ** 2))

  believed_position = (true_robot_position[0] + x_error * 2 / distance, true_robot_position[1] + y_error * 2 / distance, true_robot_position[2] + z_error * 2 / distance)

  return believed_position

def get_angle(true_robot_position, source_position, add_error = False):
  robot_position = true_robot_position
  if add_error == True:
    robot_position = get_robot_position_error(robot_position)

  delta_x = source_position[0] - robot_position[0]
  delta_y = source_position[1] - robot_position[1]
  delta_z = source_position[2] - robot_position[2]

  theta = np.arctan(delta_y / delta_x)
  phi = np.arccos(delta_z / np.sqrt((delta_x ** 2) + (delta_y ** 2) + (delta_z ** 2)))

  if add_error == True:
    angle = random.random() * 10000

    theta += 0.0872665 * np.sin(angle)
    phi += 0.0872665 * np.cos(angle)

  dx = np.sin(phi) * np.cos(theta) * (delta_x / np.abs(delta_x))
  dy = np.sin(phi) * np.sin(theta) * (delta_y / np.abs(delta_y))
  dz = np.cos(phi)

  return (dx, dy, dz)

In [7]:
source = ((random.random()*1000)%room_w, (random.random()*1000)%room_l, (random.random()*1000)%room_h)
source = (1000, 1000, 0)
robot1 = (0, 0, 0)
robot2 = (room_w, 0, 0)
robot3 = (0, room_l, 0)
robot4 = (room_w, room_l, 0)

is_error = True

positions = (get_robot_position_error(robot1), get_robot_position_error(robot2), get_robot_position_error(robot3), get_robot_position_error(robot4))
angles = (get_angle(robot1, source, is_error), get_angle(robot2, source, is_error), get_angle(robot3, source, is_error), get_angle(robot4, source, is_error))
if is_error == True:
  positions = (robot1, robot2, robot3, robot4)
  angles = (get_angle(robot1, source), get_angle(robot2, source), get_angle(robot3, source), get_angle(robot4, source))

In [ ]:
best_value = 1000;
best_position = (-1, -1, -1)

start_time = time.time()

for i in range(0, room_w, chunk_size):
  for j in range(0, room_l, chunk_size):
    for k in range(0, room_h, chunk_size):

      total_dist = 0
      for x in range(0, len(positions), 1):
        point_to_source = (i - positions[x][0], j - positions[x][1], k - positions[x][2])
        cross_product_vector = np.cross(point_to_source, angles[x])
        distance_from_line = math.sqrt(sum(pow(element, 2) for element in cross_product_vector))
        total_dist += distance_from_line

      if total_dist < best_value:
        best_value = total_dist
        best_position = (i, j, k)

end_time = time.time()

In [8]:
print(best_position)
print(source)

difference = np.sqrt(((best_position[0] - source[0]) ** 2) + ((best_position[1] - source[1]) ** 2) + ((best_position[2] - source[2]) ** 2))
print(difference)

print(end_time - start_time)

(-1, -1, -1)
(1000, 1000, 0)
1415.6281291356145
1594.6670949459076
